<a href="https://colab.research.google.com/github/nobertomaciel/PLN-ANIMA/blob/main/PLN_Analise_de_sentimentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gensim
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 17.6 MB/s eta 0:00:00


In [10]:
# ===============================================
# Exemplo completo de PLN com Classificação de Sentimentos
# ===============================================

import re
import nltk
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import string

In [11]:
# ===============================================
# 1. Recursos do NLTK
# ===============================================
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
# ===============================================
# 2. Dataset realista de avaliações
# ===============================================
# 1 = positivo, 0 = negativo
dados = {
    "avaliacao": [
        "O celular é ótimo, tem uma câmera excelente!",
        "Horrível! Que produto terrível, não recomendo a ninguém.",
        "Entrega rápida e produto de boa qualidade.",
        "A bateria dura pouco e esquenta muito. Péssimo!",
        "Muito bom, atendeu todas as minhas expectativas.",
        "Não gostei, o material parece frágil e a tela risca fácil.",
        "Superou minhas expectativas, recomendo demais!",
        "O som é baixo e a bateria acabou em poucas horas.",
        "Produto maravilhoso, desempenho excelente.",
        "Defeituoso, não funcionou desde o primeiro dia."
    ],
    "sentimento": [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
}

df = pd.DataFrame(dados)
print(df)

                                           avaliacao  sentimento
0       O celular é ótimo, tem uma câmera excelente!           1
1  Horrível! Que produto terrível, não recomendo ...           0
2         Entrega rápida e produto de boa qualidade.           1
3    A bateria dura pouco e esquenta muito. Péssimo!           0
4   Muito bom, atendeu todas as minhas expectativas.           1
5  Não gostei, o material parece frágil e a tela ...           0
6     Superou minhas expectativas, recomendo demais!           1
7  O som é baixo e a bateria acabou em poucas horas.           0
8         Produto maravilhoso, desempenho excelente.           1
9    Defeituoso, não funcionou desde o primeiro dia.           0


In [20]:
# ===============================================
# 3. Função de pré-processamento
# ===============================================
def preprocessar_texto(texto):
    texto = texto.lower()
    texto = texto.replace('\n', ' ')
    texto = re.sub(r'[^a-zA-Zà-úÀ-Ú\s]', '', texto)
    tokens = nltk.word_tokenize(texto, language='portuguese')
    tokens = [t for t in tokens if t not in string.punctuation]
    stop_words = set(stopwords.words('portuguese'))
    tokens = [t for t in tokens if t not in stop_words and len(t) > 2]
    return " ".join(tokens)

df["texto_limpo"] = df["avaliacao"].apply(preprocessar_texto)
print(df["texto_limpo"])

0                    celular ótimo câmera excelente
1       horrível produto terrível recomendo ninguém
2              entrega rápida produto boa qualidade
3               bateria dura pouco esquenta péssimo
4                    bom atendeu todas expectativas
5    gostei material parece frágil tela risca fácil
6             superou expectativas recomendo demais
7             som baixo bateria acabou poucas horas
8          produto maravilhoso desempenho excelente
9           defeituoso funcionou desde primeiro dia
Name: texto_limpo, dtype: object


In [21]:
# ===============================================
# 4. Representações vetoriais (Bag-of-Words e TF-IDF)
# ===============================================
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["texto_limpo"])
y = df["sentimento"]
print(y)
print(X)

0    1
1    0
2    1
3    0
4    1
5    0
6    1
7    0
8    1
9    0
Name: sentimento, dtype: int64
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 49 stored elements and shape (10, 43)>
  Coords	Values
  (0, 6)	0.5182909034319405
  (0, 42)	0.5182909034319405
  (0, 7)	0.5182909034319405
  (0, 16)	0.44059461896295216
  (1, 23)	0.4836062152711379
  (1, 31)	0.3596722024962039
  (1, 40)	0.4836062152711379
  (1, 34)	0.4111094652339048
  (1, 26)	0.4836062152711379
  (2, 31)	0.348545761726207
  (2, 14)	0.4686458822988574
  (2, 36)	0.4686458822988574
  (2, 4)	0.4686458822988574
  (2, 33)	0.4686458822988574
  (3, 3)	0.3911762483232489
  (3, 13)	0.4601578921271882
  (3, 29)	0.4601578921271882
  (3, 15)	0.4601578921271882
  (3, 32)	0.4601578921271882
  (4, 5)	0.5182909034319405
  (4, 1)	0.5182909034319405
  (4, 41)	0.5182909034319405
  (4, 17)	0.44059461896295216
  (5, 21)	0.3779644730092272
  (5, 25)	0.3779644730092272
  (5, 27)	0.3779644730092272
  (5, 18)	0.3779644730092272
  (5

In [ ]:
# ===============================================
# 5. Divisão de dados (treino e teste)
# ===============================================
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# ===============================================
# 6. Aplicação do Machine Learning: modelo de classificação (Regressão Logística)
# ===============================================
modelo = LogisticRegression(max_iter=1000)
modelo.fit(X_train, y_train)

In [ ]:
# ===============================================
# 7. Avaliação do modelo
# ===============================================
y_pred = modelo.predict(X_test)
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=["Negativo", "Positivo"]))

print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

In [ ]:
# ===============================================
# 8. Teste com novas avaliações
# ===============================================
novas_avaliacoes = [
    "A câmera é incrível e a bateria dura o dia todo.",
    "O produto veio quebrado e não liga mais.",
    "Entrega rápida, mas o desempenho é ruim.",
    "Excelente custo-benefício, estou muito satisfeito!"
]

# Pré-processar e transformar
novas_proc = [preprocessar_texto(a) for a in novas_avaliacoes]
X_novas = vectorizer.transform(novas_proc)
predicoes = modelo.predict(X_novas)

# Exibir resultados
print("\nResultados em novas avaliações:")
for texto, pred in zip(novas_avaliacoes, predicoes):
    sentimento = "Positivo 😀" if pred == 1 else "Negativo 😞"
    print(f"- {texto} → {sentimento}")

In [ ]:
# ===============================================
# 9. Word2Vec (para análise semântica complementar)
# ===============================================
tokens = [t.split() for t in df["texto_limpo"]]
modelo_w2v = Word2Vec(sentences=tokens, vector_size=50, window=3, min_count=1, sg=1)

print("\nPalavras mais similares a 'ótimo':")
try:
    print(modelo_w2v.wv.most_similar('ótimo'))
except KeyError:
    print("A palavra 'ótimo' não está no vocabulário.")